In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
dfdict = {}
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.splitext(filename)[0])
        df = pd.read_csv(os.path.join(dirname, filename))
        dfdict.update({os.path.splitext(filename)[0]:df})
        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

players_16
players_15
players_18
players_19
players_20
teams_and_leagues
players_17


In [2]:
dfdict["players_20"].head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,...,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,32,1987-06-24,170,72,Argentina,FC Barcelona,...,68+2,66+2,66+2,66+2,68+2,63+2,52+2,52+2,52+2,63+2
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,34,1985-02-05,187,83,Portugal,Juventus,...,65+3,61+3,61+3,61+3,65+3,61+3,53+3,53+3,53+3,61+3
2,190871,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Junior,27,1992-02-05,175,68,Brazil,Paris Saint-Germain,...,66+3,61+3,61+3,61+3,66+3,61+3,46+3,46+3,46+3,61+3
3,200389,https://sofifa.com/player/200389/jan-oblak/20/...,J. Oblak,Jan Oblak,26,1993-01-07,188,87,Slovenia,Atlético Madrid,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,183277,https://sofifa.com/player/183277/eden-hazard/2...,E. Hazard,Eden Hazard,28,1991-01-07,175,74,Belgium,Real Madrid,...,66+3,63+3,63+3,63+3,66+3,61+3,49+3,49+3,49+3,61+3


In [3]:
df15 = dfdict["players_15"]
df16 = dfdict["players_16"]
df17 = dfdict["players_17"]
df18 = dfdict["players_18"]
df19 = dfdict["players_19"]
df20 = dfdict["players_20"]

The objective now is to work with fifa 20 dataset primarily and come up with a predivtor model for player base stats.
We Begin by listing out the columns of the entire data set for fifa 20 and then select the columns that we want to use as a feature column.

In [4]:
for x in df20.columns:
    print(x,",",end=" ")

sofifa_id , player_url , short_name , long_name , age , dob , height_cm , weight_kg , nationality , club , overall , potential , value_eur , wage_eur , player_positions , preferred_foot , international_reputation , weak_foot , skill_moves , work_rate , body_type , real_face , release_clause_eur , player_tags , team_position , team_jersey_number , loaned_from , joined , contract_valid_until , nation_position , nation_jersey_number , pace , shooting , passing , dribbling , defending , physic , gk_diving , gk_handling , gk_kicking , gk_reflexes , gk_speed , gk_positioning , player_traits , attacking_crossing , attacking_finishing , attacking_heading_accuracy , attacking_short_passing , attacking_volleys , skill_dribbling , skill_curve , skill_fk_accuracy , skill_long_passing , skill_ball_control , movement_acceleration , movement_sprint_speed , movement_agility , movement_reactions , movement_balance , power_shot_power , power_jumping , power_stamina , power_strength , power_long_shots , 

In [5]:
vals = set(df20['wage_eur'])

print(vals)

{64000, 0, 405000, 21000, 85000, 170000, 42000, 255000, 63000, 340000, 20000, 84000, 105000, 41000, 62000, 19000, 83000, 40000, 125000, 61000, 210000, 18000, 82000, 39000, 60000, 145000, 17000, 81000, 38000, 59000, 16000, 80000, 165000, 37000, 235000, 250000, 58000, 15000, 79000, 36000, 100000, 185000, 57000, 78000, 14000, 355000, 35000, 99000, 120000, 56000, 205000, 77000, 13000, 290000, 34000, 98000, 55000, 140000, 76000, 12000, 225000, 97000, 33000, 54000, 75000, 11000, 160000, 96000, 32000, 565000, 245000, 53000, 330000, 74000, 10000, 31000, 95000, 180000, 52000, 265000, 73000, 9000, 94000, 30000, 115000, 51000, 200000, 72000, 8000, 285000, 93000, 29000, 370000, 50000, 135000, 71000, 7000, 220000, 92000, 28000, 49000, 70000, 6000, 155000, 91000, 27000, 240000, 48000, 69000, 5000, 26000, 90000, 175000, 47000, 68000, 4000, 89000, 25000, 110000, 46000, 195000, 67000, 3000, 24000, 88000, 45000, 130000, 66000, 2000, 215000, 23000, 87000, 300000, 44000, 65000, 1000, 470000, 150000, 86000

For the Base Stats value we will consider the field overall as the Target value and then in our second model we will be appending base our generated overall to the features and calculate the players potential

In [6]:
fifa20_features = ['age','height_cm','weight_kg','nationality','international_reputation','weak_foot','work_rate','club','skill_moves']

fifa20_potential = ['age','height_cm','weight_kg','nationality','international_reputation','weak_foot','work_rate','club','skill_moves','overall']

fifa20_value = ['age','height_cm','weight_kg','nationality','international_reputation','weak_foot','work_rate','club','skill_moves','overall','potential']

fifa20_wage = ['age','height_cm','weight_kg','nationality','international_reputation','weak_foot','work_rate','club','skill_moves','overall','potential','value_eur']

X1 = df20[fifa20_features]

X2 = df20[fifa20_potential]

X3 = df20[fifa20_value]

X4 = df20[fifa20_wage]


y1 = df20['overall']

y2 = df20['potential']

y3 = df20['value_eur']

y4 = df20['wage_eur']

In [7]:
print(set(X1['work_rate']))

{'Medium/High', 'Medium/Low', 'Low/Medium', 'High/Medium', 'High/High', 'Low/High', 'Low/Low', 'High/Low', 'Medium/Medium'}


We are now going to split the data into a train and test

From our observations of the Dataset we can clearly figure out that these columns with none integer values have some categorical values to them. For Example if a player belonged to a certain country such as Spain, or played for a club like Barcelona would automatically have a better base stats compared to a player from any of the other Asian Countries. And the work rate would also have an affect on the ability of the players.

And so by following this logic, it only makes sense to Label Encode these three columns.


One of the challenges faced while Label Encoding was that the value of Label in training data was not present in Test Data.
To overcome this, rather than splitting and then encoding. We performed Encoding and then Split the Data

In [8]:
# Get list of categorical variables
s1 = (X1.dtypes == 'object')
object_cols1 = list(s1[s1].index)

print("Categorical variables for Base Stats:")
print(object_cols1)


s2 = (X2.dtypes == 'object')
object_cols2 = list(s2[s2].index)

print("Categorical variables for Potential:")
print(object_cols2)


s3 = (X3.dtypes == 'object')
object_cols3 = list(s3[s3].index)

print("Categorical variables for Player Value:")
print(object_cols3)


s4 = (X4.dtypes == 'object')
object_cols4 = list(s4[s4].index)

print("Categorical variables for Player Wage:")
print(object_cols4)

Categorical variables for Base Stats:
['nationality', 'work_rate', 'club']
Categorical variables for Potential:
['nationality', 'work_rate', 'club']
Categorical variables for Player Value:
['nationality', 'work_rate', 'club']
Categorical variables for Player Wage:
['nationality', 'work_rate', 'club']


In [9]:
from sklearn.preprocessing import LabelEncoder

# Make copy to avoid changing original data 
# Apply label encoder to each column with categorical data
label_X1 = X1.copy()
label_X2 = X2.copy()
label_X3 = X3.copy()
label_X4 = X4.copy()
label_encoder = LabelEncoder()
for col in object_cols1:
    label_X1[col+'_features'] = label_encoder.fit_transform(X1[col])
for col in object_cols2:
    label_X2[col+'_features'] = label_encoder.fit_transform(X2[col])
for col in object_cols3:
    label_X3[col+'_features'] = label_encoder.fit_transform(X3[col])
for col in object_cols4:
    label_X4[col+'_features'] = label_encoder.fit_transform(X4[col])

In [10]:
countryname = set(X1['nationality'])
countrycode = set(label_X1['nationality_features'])
countrydict = dict(zip(countryname,countrycode))

wrname = set(X1['work_rate'])
wrcode = set(label_X1['work_rate_features'])
wrdict = dict(zip(wrname,wrcode))

clubname = set(X1['club'])
clubcode = set(label_X1['club_features'])
clubdict = dict(zip(clubname,clubcode))


In [11]:
wrname

{'High/High',
 'High/Low',
 'High/Medium',
 'Low/High',
 'Low/Low',
 'Low/Medium',
 'Medium/High',
 'Medium/Low',
 'Medium/Medium'}

In [12]:
wrdict

{'Medium/High': 0,
 'Medium/Low': 1,
 'Low/Medium': 2,
 'High/Medium': 3,
 'High/High': 4,
 'Low/High': 5,
 'Low/Low': 6,
 'High/Low': 7,
 'Medium/Medium': 8}

In [13]:
clubdict

{'Atalanta': 0,
 'Aalborg BK': 1,
 'US Salernitana 1919': 2,
 'Gillingham': 3,
 'Napoli': 4,
 'Ettifaq FC': 5,
 '1. FC Union Berlin': 6,
 'Argentinos Juniors': 7,
 'Patriotas Boyacá FC': 8,
 'Sepsi OSK': 9,
 'AFC Eskilstuna': 10,
 'Goiás': 11,
 'Sint-Truidense VV': 12,
 'Jaguares de Córdoba': 13,
 'Shanghai Greenland Shenhua FC': 14,
 'Atlético de San Luis': 15,
 'Junior FC': 16,
 'Famalicão': 17,
 'VVV-Venlo': 18,
 'RSC Anderlecht': 19,
 'Sassuolo': 20,
 'Arka Gdynia': 21,
 'FC Red Bull Salzburg': 22,
 'CD Palestino': 23,
 'Once Caldas': 24,
 'Shamrock Rovers': 25,
 'Denizlispor': 26,
 'SL Benfica': 27,
 'SV Sandhausen': 28,
 'Motherwell': 29,
 'IK Sirius': 30,
 'MKE Ankaragücü': 31,
 'FC Cincinnati': 32,
 'Luton Town': 33,
 'Grenoble Foot 38': 34,
 'Kristiansund BK': 35,
 'Newcastle Jets': 36,
 'Le Havre AC': 37,
 'Godoy Cruz': 38,
 'Al Nassr': 39,
 'PEC Zwolle': 40,
 'FC Paços de Ferreira': 41,
 'Swansea City': 42,
 'Jeonbuk Hyundai Motors': 43,
 'FC Ingolstadt 04': 44,
 'Randers FC

In [14]:
label_X1.head()


,age,height_cm,weight_kg,nationality,international_reputation,weak_foot,work_rate,club,skill_moves,nationality_features,work_rate_features,club_features
0,32,170,72,Argentina,5,4,Medium/Low,FC Barcelona,4,5,7,224
1,34,187,83,Portugal,5,4,High/Low,Juventus,5,121,1,350
2,27,175,68,Brazil,5,5,High/Medium,Paris Saint-Germain,5,18,2,462
3,26,188,87,Slovenia,3,3,Medium/Medium,Atlético Madrid,1,133,8,63
4,28,175,74,Belgium,4,4,High/Medium,Real Madrid,4,12,2,505


In [15]:
label_X2.head()

,age,height_cm,weight_kg,nationality,international_reputation,weak_foot,work_rate,club,skill_moves,overall,nationality_features,work_rate_features,club_features
0,32,170,72,Argentina,5,4,Medium/Low,FC Barcelona,4,94,5,7,224
1,34,187,83,Portugal,5,4,High/Low,Juventus,5,93,121,1,350
2,27,175,68,Brazil,5,5,High/Medium,Paris Saint-Germain,5,92,18,2,462
3,26,188,87,Slovenia,3,3,Medium/Medium,Atlético Madrid,1,91,133,8,63
4,28,175,74,Belgium,4,4,High/Medium,Real Madrid,4,91,12,2,505


In [16]:
label_X3.head()

,age,height_cm,weight_kg,nationality,international_reputation,weak_foot,work_rate,club,skill_moves,overall,potential,nationality_features,work_rate_features,club_features
0,32,170,72,Argentina,5,4,Medium/Low,FC Barcelona,4,94,94,5,7,224
1,34,187,83,Portugal,5,4,High/Low,Juventus,5,93,93,121,1,350
2,27,175,68,Brazil,5,5,High/Medium,Paris Saint-Germain,5,92,92,18,2,462
3,26,188,87,Slovenia,3,3,Medium/Medium,Atlético Madrid,1,91,93,133,8,63
4,28,175,74,Belgium,4,4,High/Medium,Real Madrid,4,91,91,12,2,505


In [17]:
label_X4.head()

,age,height_cm,weight_kg,nationality,international_reputation,weak_foot,work_rate,club,skill_moves,overall,potential,value_eur,nationality_features,work_rate_features,club_features
0,32,170,72,Argentina,5,4,Medium/Low,FC Barcelona,4,94,94,95500000,5,7,224
1,34,187,83,Portugal,5,4,High/Low,Juventus,5,93,93,58500000,121,1,350
2,27,175,68,Brazil,5,5,High/Medium,Paris Saint-Germain,5,92,92,105500000,18,2,462
3,26,188,87,Slovenia,3,3,Medium/Medium,Atlético Madrid,1,91,93,77500000,133,8,63
4,28,175,74,Belgium,4,4,High/Medium,Real Madrid,4,91,91,90000000,12,2,505


In [18]:
from sklearn.model_selection import train_test_split

# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.
#lX1 = label_X1['age','height_cm','weight_kg','international_reputation','weak_foot','skill_moves','nationality_features','work_rate_features','club_features']
#lX2 = label_X2['age','height_cm','weight_kg','international_reputation','weak_foot','skill_moves','overall','nationality_features','work_rate_features','club_features']

lX1 = label_X1.copy()
lX2 = label_X2.copy()
lX3 = label_X3.copy()
lX4 = label_X4.copy()
lX1.drop(['nationality','work_rate','club'],axis=1,inplace=True)
lX2.drop(['nationality','work_rate','club'],axis=1,inplace=True)
lX3.drop(['nationality','work_rate','club'],axis=1,inplace=True)
lX4.drop(['nationality','work_rate','club'],axis=1,inplace=True)
train_X1, val_X1, train_y1, val_y1 = train_test_split(lX1, y1, random_state = 0)
train_X2, val_X2, train_y2, val_y2 = train_test_split(lX2, y2, random_state = 0)
train_X3, val_X3, train_y3, val_y3 = train_test_split(lX3, y3, random_state = 0)
train_X4, val_X4, train_y4, val_y4 = train_test_split(lX4, y4, random_state = 0)

In [19]:
from xgboost import XGBRegressor

basestats_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
basestats_model.fit(train_X1, train_y1)

potential_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
potential_model.fit(train_X2, train_y2)

playervalue_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
playervalue_model.fit(train_X3, train_y3)

playerwage_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
playerwage_model.fit(train_X4, train_y4)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [20]:
from sklearn.metrics import mean_absolute_error

base_predictions = basestats_model.predict(val_X1)
print("Mean Absolute Error: " + str(mean_absolute_error(base_predictions, val_y1)))

potential_predictions = potential_model.predict(val_X2)
print("Mean Absolute Error: " + str(mean_absolute_error(potential_predictions, val_y2)))

playervalue_predictions = playervalue_model.predict(val_X3)
print("Mean Absolute Error: " + str(mean_absolute_error(playervalue_predictions, val_y3)))

playerwage_predictions = playerwage_model.predict(val_X4)
print("Mean Absolute Error: " + str(mean_absolute_error(playerwage_predictions, val_y4)))

Mean Absolute Error: 2.902640110241245
Mean Absolute Error: 1.0299911582496026
Mean Absolute Error: 171226.22101193352
Mean Absolute Error: 2365.987637716936


In [21]:
val_y3.head()


10114    825000
13805    290000
14836    375000
8164     975000
6608     850000
Name: value_eur, dtype: int64

In [22]:
print("\n",playervalue_predictions[0:5])


 [820447.06 270492.38 406056.25 949083.5  928597.4 ]


Now that the base and potential models are ready we are going to experiment with new values and see if they can be used to predict a model outcome

In [23]:
for val in train_X1:
    print(val)

age
height_cm
weight_kg
international_reputation
weak_foot
skill_moves
nationality_features
work_rate_features
club_features


In [24]:
for val in train_X2:
    print(val)

age
height_cm
weight_kg
international_reputation
weak_foot
skill_moves
overall
nationality_features
work_rate_features
club_features


In [25]:
for val in train_X3:
    print(val)

age
height_cm
weight_kg
international_reputation
weak_foot
skill_moves
overall
potential
nationality_features
work_rate_features
club_features


In [26]:
for val in train_X4:
    print(val)

age
height_cm
weight_kg
international_reputation
weak_foot
skill_moves
overall
potential
value_eur
nationality_features
work_rate_features
club_features


Now we can try and experiment by accepting values from the user and see how his features stack against the created model


In [27]:
import math

print("First Name: \t",end='')
FirstName = input()
print("Last Name:\t",end='')
LastName = input()
print("Enter the Age of the player (between 18 to 45):\t",end='')
a = int(input())
print("Enter the Player's height in centimeters:\t",end='')
h = int(input())
print("Enter the Player's weight in Kgs:\t",end='')
w = int(input())
print("Enter the Player's international reputation (1 to 5) \n [ranging from 1-Novice to 5-Gifted]:\t",end='')
ir = int(input())
print("Enter the Player's weak foot ability (1 to 5) \n [ranging from 1-Novice to 5-Gifted]:\t",end='')
wf = int(input())
print("Enter the Player's skill moves ability (1 to 5) \n [ranging from 1-Novice to 5-Gifted]:\t",end='')
sm = int(input())
print("Enter the Player's nationality:\t",end='')
nationality = input()
print("Enter the Player's work rate for Attack and Defence in the format \"att/def\" \n The work rate has to be any one for the following three (High,Medium,Low)\t",end='')
workrate = input()
#High/Low
print("Enter the Name of the Current Club:\t",end='')
club = input()

n = int(countrydict[nationality])
wr = int(wrdict[workrate])
c = int(clubdict[club])


lst1 = [[int(a),int(h),int(w),int(ir),int(wf),int(sm),int(n),int(wr),int(c)]]
l1 = pd.DataFrame(lst1,columns=train_X1.columns)
overall = math.floor(basestats_model.predict(l1))
baseerror = str(round(mean_absolute_error(base_predictions, val_y1)/2,2))

lst2 = lst1
lst2[0].insert(6,math.floor(overall))
l2 = pd.DataFrame(lst2,columns=train_X2.columns)
potential = math.floor(potential_model.predict(l2))
potentialerror = str(round(mean_absolute_error(potential_predictions, val_y2)/2,2))

lst3 = lst2
lst3[0].insert(7,math.floor(potential))
l3 = pd.DataFrame(lst3,columns=train_X3.columns)
playerval = math.floor(playervalue_model.predict(l3))
vallen = len(str(playerval))
if (vallen>6):
    vl = round((playerval/1000000),2)
    strval = str(vl) + " million"
elif (vallen <= 6 and len > 3):
    vl = round((playerval/1000),2)
    strval = str(vl) + " thousand"
else:
    strval = str(playerval)
valerror = str(round(mean_absolute_error(playervalue_predictions, val_y3)/2,2))

lst4 = lst3
lst4[0].insert(8,math.floor(playerval))
l4 = pd.DataFrame(lst4,columns=train_X4.columns)
playerwage = math.floor(playerwage_model.predict(l4))
wageerror = str(round(mean_absolute_error(playerwage_predictions, val_y4)/2,2))



print("\n\nPLAYER CARD \n\nName: {0} {1}\tAge: {2}\nNationality: {3}\tClub: {4}".format(FirstName,LastName,a,nationality,club))
print("Height(cms): {0}\tWeight(kgs): {1}".format(h,w))
print("\n\nPLAYER PREDICTED STATS \n\n")
print("Player Base Stats:\t {0} with Margin of (+/-{1})".format(overall,baseerror))
print("Player Future Potential:\t {0} with Margin of (+/-{1})".format(potential,potentialerror))
print("Player Est. Value (in Eur):\t {0} with Margin of (+/-{1})".format(strval,valerror))
print("Player Est. Wage (in Eur):\t {0} with Margin of (+/-{1})".format(playerwage,wageerror))


First Name: 	

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.